In [ ]:
def crear_video_dinamico(rutas_imagenes, rutas_audios):
    videos_individuales = []

    for i in range(len(rutas_imagenes)):
        output_clip = f"static/videos/clip_{i}.mp4"
        subprocess.run([
            "ffmpeg", "-y",
            "-loop", "1", "-i", rutas_imagenes[i],
            "-i", rutas_audios[i],
            "-c:v", "libx264", "-tune", "stillimage",
            "-c:a", "aac", "-b:a", "192k",
            "-pix_fmt", "yuv420p",
            "-shortest", "-movflags", "+faststart",
            output_clip
        ])
        videos_individuales.append(output_clip)

    lista_txt = "static/videos/lista_videos.txt"
    with open(lista_txt, "w") as f:
        for v in videos_individuales:
            f.write(f"file '{v}'\n")

    video_final = "static/videos/curso_final.mp4"
    subprocess.run([
        "ffmpeg", "-y", "-f", "concat", "-safe", "0", "-i", lista_txt,
        "-c", "copy", "-movflags", "+faststart", video_final
    ])

    return video_final


: 

In [ ]:
@app.route('/test_openai_connection', methods=['GET'])
def test_openai_connection():
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo",
            prompt="Hello, OpenAI!",
            max_tokens=5
        )
        return f"OpenAI connection successful! Response: {response.choices[0].text}"
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
def generate_image(prompt, id):
    result = openai.Image.create(prompt=prompt, n=1, size="1024x1024")
    image_url = result['data'][0]['url']
    image_path = os.path.join("static/imagenes", f"{id}.png")
    response = requests.get(image_url, timeout=30)
    if response.status_code == 200:
        with open(image_path, "wb") as f:
            f.write(response.content)
    return image_path

In [ ]:
def generate_audio(texto, id):
    """
    Genera un MP3 desde un texto con gTTS, lo guarda en static/audios/{id}.mp3
    y comprueba en consola que no esté vacío.
    """
    audio_file = os.path.join("static", "audios", f"{id}.mp3")
    tts = gTTS(text=texto, lang='es')
    tts.save(audio_file)

    # DEBUG: comprueba que realmente se escribió algo
    size = os.path.getsize(audio_file)
    print(f"[DEBUG] Audio guardado en {audio_file}, {size} bytes")

    return audio_file

# Función para generar video con FFmpeg
def generar_video_con_ffmpeg(imagen, audio, output_video):
    audio_duration = get_audio_duration(audio)
    command = [
        'ffmpeg', '-y', '-loop', '1', '-framerate', '2', '-t', str(audio_duration),
        '-i', imagen, '-i', audio, '-c:v', 'libx264', '-preset', 'veryfast',
        '-c:a', 'aac', '-b:a', '192k', '-pix_fmt', 'yuv420p', '-shortest', output_video
    ]
    subprocess.run(command, check=True)
